In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
import seaborn as sns

# Methodology
1. Use FY18, FY19, and FY20 data
2. Match majors to departments - use FY20 departments
3. Majors that do match - assume them to have been discontinued
4. Make a list of any new majors added in FY21 and estimate them separately, using averages or something.
5. Use Faculty Data from Delaware report to calculate department level revenues 
6. Keep the net revenues by department from step 5 separate. 
7. Take out the Fall term by each year and on department level aggregate other predictor variables - like college, scch, enrollment, and so forth. 
8. Create a training set for FY18 and FY19, and test for FY20 - identify algorithms best suited for less observations and more predictor variables. If not, use bootstrapping. 
9. Using FY18, FY19, and FY20 as training data, predict for FY21. 
10. Match up against budgeted expenses to get to profitability. 
11. Also use 3 year averages and 10% differentials to create ranges around the predicted profitability.

**NOTES**
1. This analysis does not take into account HE Payments

# Links

https://medium.com/@remycanario17/log-f-m-m-logit-the-best-classification-algorithm-for-small-datasets-fc92fd95bc58

https://towardsdatascience.com/breaking-the-curse-of-small-datasets-in-machine-learning-part-1-36f28b0c044d

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0224365

https://www.nature.com/articles/s41524-018-0081-z

https://www.investopedia.com/articles/financial-theory/11/using-genetic-algorithms-forecast-financial-markets.asp

# FY18

In [95]:
fy18 = pd.read_excel('../1_data/Student Data 16F to 20S.xlsx', sheet_name='17F 18S')
fy18.groupby(['CWID', 'CRS_ID', 'TERM']).ngroups - fy18['CWID'].count()

0

In [96]:
fy18['TERM'].value_counts(dropna=False)

2017F    19632
2018S    18622
Name: TERM, dtype: int64

In [97]:
fy18.columns

Index(['TERM', 'CRS_ID', 'CWID', 'Key', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'STU_SEC_COL', 'STU_SEC_MAJ', 'STU_SEC_DEG', 'SEM_HOURS', 'TUITSEM1', 'FEES', 'MFEES', 'LOY_SCHOL', 'LOY_ENDOW', 'LOY_DISC1', 'LOY_DISC2', 'LOY_DISC3', 'REDUC_DISC', 'REL_DISC', 'TUIT_WAIV', 'FIN_AT_HRS', 'CRS_AT_HRS', 'STATUS', 'CRS_COL', 'AOC', 'EXCH_STA', 'CRS_SECTION', 'CRS_AREA', 'CRS_SUBJ', 'CRS_LEV', 'COURS_LEV', 'CRHR_CHARGE', 'NET_CRHR_CHARGE', 'GROSS_CRS_REV', 'NET_CRS_REVENUE', 'SCHOL_CRHR_CHG', 'DISC_CRHR_CHG'], dtype='object')

In [98]:
fy18.head()

,TERM,CRS_ID,CWID,Key,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,STU_SEC_COL,STU_SEC_MAJ,STU_SEC_DEG,SEM_HOURS,TUITSEM1,FEES,MFEES,LOY_SCHOL,LOY_ENDOW,LOY_DISC1,LOY_DISC2,LOY_DISC3,REDUC_DISC,REL_DISC,TUIT_WAIV,FIN_AT_HRS,CRS_AT_HRS,STATUS,CRS_COL,AOC,EXCH_STA,CRS_SECTION,CRS_AREA,CRS_SUBJ,CRS_LEV,COURS_LEV,CRHR_CHARGE,NET_CRHR_CHARGE,GROSS_CRS_REV,NET_CRS_REVENUE,SCHOL_CRHR_CHG,DISC_CRHR_CHG
0,2018S,NURSG805W02,10002754,NURSG805W022018S,CNH,FNP,MSN,GFR,G,E,,,,6.0,4908.0,225.0,0,0,0.0,0,0.0,0.0,0.0,0,0.0,3.0,3.0,E,CNH,,,W02,G805,NURS,805,G,855.50,855.50,2566.50,2566.50,0.0,0.0
1,2018S,NURSG835W02,10002754,NURSG835W022018S,CNH,FNP,MSN,GFR,G,E,,,,6.0,4908.0,225.0,0,0,0.0,0,0.0,0.0,0.0,0,0.0,3.0,3.0,E,CNH,,,W02,G835,NURS,835,G,855.50,855.50,2566.50,2566.50,0.0,0.0
2,2018S,HISTT122004,10015160,HISTT1220042018S,CAS,NDRA,ND,NDU,U,E,,,,3.0,3219.0,401.5,0,0,0.0,0,0.0,0.0,-3219.0,0,0.0,3.0,3.0,E,CAS,,,004,T122,HIST,122,U,1206.83,133.83,3620.49,401.49,0.0,-1073.0
3,2018S,DECSB893001,10020635,DECSB8930012018S,BU,NDGB,ND,NDG,G,E,,,,3.0,3015.0,56.5,0,0,0.0,0,0.0,0.0,0.0,0,-3015.0,3.0,3.0,E,BU,,,001,B893,DECS,893,G,1023.83,18.83,3071.49,56.49,0.0,-1005.0
4,2018S,MUGNO268W02,10038247,MUGNO268W022018S,CAS,ENGL,BA,SR,U,A,,,,3.0,3219.0,401.5,275,0,0.0,0,0.0,0.0,0.0,0,0.0,3.0,3.0,E,MA,,,W02,O268,MUGN,268,U,1206.83,1206.83,3620.49,3620.49,0.0,0.0


In [99]:
fy18 = fy18[['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT',
            'CRS_AT_HRS', 'STATUS', 'CRS_COL', 'NET_CRS_REVENUE']]
fy18.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,STATUS,CRS_COL,NET_CRS_REVENUE
0,2018S,NURSG805W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,E,CNH,2566.50
1,2018S,NURSG835W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,E,CNH,2566.50
2,2018S,HISTT122004,10015160,CAS,NDRA,ND,NDU,U,E,3.0,E,CAS,401.49
3,2018S,DECSB893001,10020635,BU,NDGB,ND,NDG,G,E,3.0,E,BU,56.49
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,3.0,E,MA,3620.49


In [100]:
fy18['cohort_compare']=np.where(fy18['STU_COHORT']==fy18['STATUS'],1,0)
fy18['cohort_compare'].value_counts(dropna=False)

1    33296
0     4958
Name: cohort_compare, dtype: int64

In [101]:
fy18[fy18['cohort_compare']==0].head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,STATUS,CRS_COL,NET_CRS_REVENUE,cohort_compare
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,3.0,E,MA,3620.49,0
5,2018S,LIBRG210WA1,10038858,CAE,HUMA,BLS,SR,U,A,3.0,E,ID,1464.99,0
11,2018S,TRINE497001,10056192,CE,TRIN,CTI,CED,C,A,3.0,E,CE,577.50,0
17,2018S,ITALA101001,10059175,CAS,NDUP,ND,NDU,U,A,3.0,E,CAS,3620.49,0
18,2017F,ITALA100001,10059175,CAS,NDUP,ND,NDU,U,A,3.0,E,CAS,3620.49,0


In [102]:
fy18['STATUS'].value_counts(dropna=False)

E    38254
Name: STATUS, dtype: int64

In [103]:
fy18.drop(['STATUS', 'cohort_compare'], inplace=True, axis=1)
fy18.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE
0,2018S,NURSG805W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50
1,2018S,NURSG835W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50
2,2018S,HISTT122004,10015160,CAS,NDRA,ND,NDU,U,E,3.0,CAS,401.49
3,2018S,DECSB893001,10020635,BU,NDGB,ND,NDG,G,E,3.0,BU,56.49
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,3.0,MA,3620.49


In [104]:
fy18['STU_MAJ'].value_counts(dropna=False)

CIVD    2536
MUIN    2532
BIOS    1785
PSYC    1595
COM     1441
MUTY    1326
PCM     1135
MKT     1033
MGT      978
CMPR     828
CMJR     799
POLS     790
FIN      740
ENGW     724
ACC      708
MUED     707
DSGN     663
JAZZ     661
DFM      630
CHEB     626
PSYP     591
INTB     569
CRIM     565
PERV     552
GENS     504
CIVE     497
MUSC     478
MUIM     441
MUIB     419
NURL     418
THEA     409
THMT     382
FNP      380
CRJU     376
CMAD     374
CNSL     366
STUA     357
PERI     332
GSB      324
ECON     306
MBAP     304
ENVB     300
HIST     280
EXUP     277
SOCI     266
ENGL     265
MUCP     230
SOSJ     228
LIMO     222
NURB     216
ENGM     208
PHYS     208
THTR     196
MBAF     189
CHFS     185
PHPL     176
ENVS     165
BUAN     157
HSPL     140
COSC     140
NURM     132
LIMX     129
CHEM     114
PHIL     114
CVPT     112
MATH     101
IEN       97
PHYE      95
CIS       94
TRIN      94
SOCL      86
BPPH      82
NURS      78
TRCN      72
CSGP      71
MCJ       67
TEAC      67

Remove majors that have been discontinued. No point projecting over them. See file Student Ratios by Majors.xlsx in FY20, sheet FY18.

In [105]:
crosswalk = pd.read_excel('../../../FY20 Data/Course Data/Student Majors and Department Crosswalk.xlsx', 
                          usecols = ['STU_MAJ', 'Department', 'College'])
crosswalk = crosswalk[pd.notna(crosswalk['Department'])]
crosswalk.tail()

,STU_MAJ,Department,College
132,TRCN,CAS,CAS
133,TRDP,Nursing,CNH
134,TRPP,Law,LAW
135,TRUP,CAS,CAS
136,UEMP,Music Industry,CMM


In [106]:
crosswalk['STU_MAJ'].count() - crosswalk['STU_MAJ'].nunique()

0

In [107]:
fy18 = fy18.merge(crosswalk, on='STU_MAJ', how='left', validate='m:1')

In [108]:
fy18.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE,Department,College
0,2018S,NURSG805W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50,Nursing,CNH
1,2018S,NURSG835W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50,Nursing,CNH
2,2018S,HISTT122004,10015160,CAS,NDRA,ND,NDU,U,E,3.0,CAS,401.49,CAS,CAS
3,2018S,DECSB893001,10020635,BU,NDGB,ND,NDG,G,E,3.0,BU,56.49,BU,BU
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,3.0,MA,3620.49,English,CAS


In [109]:
fy18['TERM'][pd.isna(fy18['Department'])].count()

359

In [110]:
fy18['STU_MAJ'][pd.isna(fy18['Department'])].value_counts(dropna=False)

TRIN    94
PHYP    54
CNHS    46
SOSC    36
MATC    24
CNVD    21
CNAE    17
CNEI    14
CNSG    11
CNEJ    11
CNIR     8
CNID     7
CNCF     6
CLLT     5
NDES     4
CLGK     1
Name: STU_MAJ, dtype: int64

In [111]:
fy18['TERM'][pd.isna(fy18['Department'])].count()/fy18['TERM'].count() #less than 1% of courses - ok

0.009384639514821979

In [112]:
fy18 = fy18[pd.notna(fy18['Department'])]

In [113]:
fy18.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE,Department,College
0,2018S,NURSG805W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50,Nursing,CNH
1,2018S,NURSG835W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50,Nursing,CNH
2,2018S,HISTT122004,10015160,CAS,NDRA,ND,NDU,U,E,3.0,CAS,401.49,CAS,CAS
3,2018S,DECSB893001,10020635,BU,NDGB,ND,NDG,G,E,3.0,BU,56.49,BU,BU
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,3.0,MA,3620.49,English,CAS


In [114]:
fy18['NET_CRS_REVENUE'].sum()

54083561.94

In [115]:
fy18.to_excel('../3_res/FY18_Revenues.xlsx', index=False)

# FY 19

In [116]:
fy19 = pd.read_excel('../1_data/Student Data 16F to 20S.xlsx', sheet_name='18F 19S')
fy19.groupby(['CWID', 'CRS_ID', 'TERM']).ngroups - fy19['CWID'].count()

0

In [117]:
fy19['TERM'].value_counts(dropna=False)

2018F    21300
2019S    20138
Name: TERM, dtype: int64

In [118]:
fy19.columns

Index(['TERM', 'CRS_ID', 'CWID', 'Key', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'STU_SEC_COL', 'STU_SEC_MAJ', 'STU_SEC_DEG', 'SEM_HOURS', 'TUITSEM1', 'FEES', 'MFEES', 'LOY_SCHOL', 'LOY_ENDOW', 'LOY_DISC1', 'LOY_DISC2', 'LOY_DISC3', 'REDUC_DISC', 'REL_DISC', 'TUIT_WAIV', 'FIN_AT_HRS', 'CRS_AT_HRS', 'STATUS', 'CRS_COL', 'AOC', 'EXCH_STA', 'CRS_SECTION', 'CRS_AREA', 'CRS_SUBJ', 'CRS_LEV', 'COURS_LEV', 'CRHR_CHARGE', 'NET_CRHR_CHARGE', 'GROSS_CRS_REV', 'NET_CRS_REVENUE', 'SCHOL_CRHR_CHG', 'DISC_CRHR_CHG'], dtype='object')

In [119]:
fy19.head()

,TERM,CRS_ID,CWID,Key,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,STU_SEC_COL,STU_SEC_MAJ,STU_SEC_DEG,SEM_HOURS,TUITSEM1,FEES,MFEES,LOY_SCHOL,LOY_ENDOW,LOY_DISC1,LOY_DISC2,LOY_DISC3,REDUC_DISC,REL_DISC,TUIT_WAIV,FIN_AT_HRS,CRS_AT_HRS,STATUS,CRS_COL,AOC,EXCH_STA,CRS_SECTION,CRS_AREA,CRS_SUBJ,CRS_LEV,COURS_LEV,CRHR_CHARGE,NET_CRHR_CHARGE,GROSS_CRS_REV,NET_CRS_REVENUE,SCHOL_CRHR_CHG,DISC_CRHR_CHG
0,2019S,ACCTB202WA1,10296526,ACCTB202WA12019S,MA,THMT,BA,SR,U,E,,,,21.0,19063.0,783.0,100,-9000,0,-1550,0.0,0.0,0.0,0,0.0,3.0,3.0,E,BU,,,WA1,B202,ACCT,202,U,945.05,442.67,2835.15,1328.01,-428.57,-73.81
1,2019S,ACCTB202WA1,10294008,ACCTB202WA12019S,MA,THTR,BA,SO,U,E,,,,21.0,19063.0,783.0,0,-10500,0,0,0.0,0.0,0.0,0,0.0,3.0,3.0,E,BU,,,WA1,B202,ACCT,202,U,945.05,445.05,2835.15,1335.15,-500.00,0.00
2,2019S,ACCTB202WA1,10293554,ACCTB202WA12019S,MA,STUA,BFA,JR,U,E,,,,21.0,19063.0,783.0,175,-6267,-1733,0,0.0,0.0,0.0,0,0.0,3.0,3.0,E,BU,,T,WA1,B202,ACCT,202,U,945.05,564.10,2835.15,1692.30,-298.43,0.00
3,2019S,ACCTB202WA1,10273719,ACCTB202WA12019S,MA,MUIN,BS,SR,U,E,,,,22.0,19063.0,783.0,200,-8500,0,0,0.0,0.0,0.0,0,0.0,3.0,3.0,E,BU,,,WA1,B202,ACCT,202,U,902.09,515.73,2706.27,1547.19,-386.36,0.00
4,2019S,ACCTB202WA1,10282986,ACCTB202WA12019S,MA,MUIN,BS,SR,U,E,,,,13.0,19063.0,783.0,200,-6500,0,-5000,0.0,0.0,0.0,0,0.0,3.0,3.0,E,BU,,,WA1,B202,ACCT,202,U,1526.62,642.00,4579.86,1926.00,-500.00,-384.62


In [120]:
fy19 = fy19[['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT',
            'CRS_AT_HRS', 'CRS_COL', 'NET_CRS_REVENUE']]
fy19.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE
0,2019S,ACCTB202WA1,10296526,MA,THMT,BA,SR,U,E,3.0,BU,1328.01
1,2019S,ACCTB202WA1,10294008,MA,THTR,BA,SO,U,E,3.0,BU,1335.15
2,2019S,ACCTB202WA1,10293554,MA,STUA,BFA,JR,U,E,3.0,BU,1692.30
3,2019S,ACCTB202WA1,10273719,MA,MUIN,BS,SR,U,E,3.0,BU,1547.19
4,2019S,ACCTB202WA1,10282986,MA,MUIN,BS,SR,U,E,3.0,BU,1926.00


Remove majors that have been discontinued. No point projecting over them. See file Student Ratios by Majors.xlsx in FY20, sheet FY18.

In [121]:
fy19 = fy19.merge(crosswalk, on='STU_MAJ', how='left', validate='m:1')

In [122]:
fy19.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE,Department,College
0,2019S,ACCTB202WA1,10296526,MA,THMT,BA,SR,U,E,3.0,BU,1328.01,Theatre,CMM
1,2019S,ACCTB202WA1,10294008,MA,THTR,BA,SO,U,E,3.0,BU,1335.15,Theatre,CMM
2,2019S,ACCTB202WA1,10293554,MA,STUA,BFA,JR,U,E,3.0,BU,1692.30,Studio Art,CMM
3,2019S,ACCTB202WA1,10273719,MA,MUIN,BS,SR,U,E,3.0,BU,1547.19,Music Industry,CMM
4,2019S,ACCTB202WA1,10282986,MA,MUIN,BS,SR,U,E,3.0,BU,1926.00,Music Industry,CMM


In [123]:
fy19['TERM'][pd.isna(fy19['Department'])].count()

74

In [124]:
fy19['STU_MAJ'][pd.isna(fy19['Department'])].value_counts(dropna=False)

CNHS    29
SOSC    13
NDES    12
CNIR    10
CNAE     9
CNCF     1
Name: STU_MAJ, dtype: int64

In [125]:
fy19['TERM'][pd.isna(fy19['Department'])].count()/fy19['TERM'].count() #less than 0.2% of courses - ok

0.0017858004729958009

In [126]:
fy19 = fy19[pd.notna(fy19['Department'])]

In [127]:
fy19.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE,Department,College
0,2019S,ACCTB202WA1,10296526,MA,THMT,BA,SR,U,E,3.0,BU,1328.01,Theatre,CMM
1,2019S,ACCTB202WA1,10294008,MA,THTR,BA,SO,U,E,3.0,BU,1335.15,Theatre,CMM
2,2019S,ACCTB202WA1,10293554,MA,STUA,BFA,JR,U,E,3.0,BU,1692.30,Studio Art,CMM
3,2019S,ACCTB202WA1,10273719,MA,MUIN,BS,SR,U,E,3.0,BU,1547.19,Music Industry,CMM
4,2019S,ACCTB202WA1,10282986,MA,MUIN,BS,SR,U,E,3.0,BU,1926.00,Music Industry,CMM


In [128]:
fy19['NET_CRS_REVENUE'].sum()

59189058.20000001

In [129]:
fy19.to_excel('../3_res/FY19_Revenues.xlsx', index=False)

# FY20

In [130]:
fy20 = pd.read_excel('../1_data/Student Data 16F to 20S.xlsx', sheet_name='19F 20S')
fy20.groupby(['CWID', 'CRS_ID', 'TERM']).ngroups - fy20['CWID'].count()

0

In [131]:
fy20['TERM'].value_counts(dropna=False)

2019F    21444
2020S    20631
Name: TERM, dtype: int64

In [132]:
fy20.columns

Index(['TERM', 'CRS_ID', 'Key', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'STU_SEC_COL', 'STU_SEC_MAJ', 'STU_SEC_DEG', 'SEM_HOURS', 'TUITSEM1', 'FEES', 'MFEES', 'LOY_SCHOL', 'LOY_ENDOW', 'LOY_DISC1', 'LOY_DISC2', 'LOY_DISC3', 'REDUC_DISC', 'REL_DISC', 'TUIT_WAIV', 'CRS_ID.1', 'FIN_AT_HRS', 'CRS_AT_HRS', 'STATUS', 'CRS_COL', 'AOC', 'EXCH_STA', 'CRS_SECTION', 'CRS_AREA', 'CRS_SUBJ', 'CRS_LEV', 'COURS_LEV', 'CRHR_CHARGE', 'NET_CRHR_CHARGE', 'GROSS_CRS_REV', 'NET_CRS_REVENUE', 'SCHOL_CRHR_CHG', 'DISC_CRHR_CHG'], dtype='object')

In [133]:
fy20.head()

,TERM,CRS_ID,Key,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,STU_SEC_COL,STU_SEC_MAJ,STU_SEC_DEG,SEM_HOURS,TUITSEM1,FEES,MFEES,LOY_SCHOL,LOY_ENDOW,LOY_DISC1,LOY_DISC2,LOY_DISC3,REDUC_DISC,REL_DISC,TUIT_WAIV,CRS_ID.1,FIN_AT_HRS,CRS_AT_HRS,STATUS,CRS_COL,AOC,EXCH_STA,CRS_SECTION,CRS_AREA,CRS_SUBJ,CRS_LEV,COURS_LEV,CRHR_CHARGE,NET_CRHR_CHARGE,GROSS_CRS_REV,NET_CRS_REVENUE,SCHOL_CRHR_CHG,DISC_CRHR_CHG
0,2020S,ACCTB202001,2020SBU ACCTB202001,10295572,MA,MUIN,BS,SR,U,E,,,,19.0,19463.0,833.0,250.0,-9500,0,-3950,0.0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1068.21,360.32,3204.63,1080.96,-500.00,-207.89
1,2020S,ACCTB202001,2020SBU ACCTB202001,10310914,BU,MGT,BBA,SO,U,E,,,,15.0,19463.0,833.0,250.0,-8000,-3470,-2730,0.0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1353.07,406.40,4059.21,1219.20,-533.33,-182.00
2,2020S,ACCTB202001,2020SBU ACCTB202001,10309131,BU,GSB,UND,SO,U,E,,,,15.0,19463.0,833.0,250.0,-8500,0,-6450,0.0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1353.07,356.40,4059.21,1069.20,-566.67,-430.00
3,2020S,ACCTB202001,2020SBU ACCTB202001,10307124,BU,MGT,BBA,SO,U,E,,,,12.0,19463.0,833.0,250.0,-5000,0,-12400,0.0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1691.33,241.33,5073.99,723.99,-416.67,-1033.33
4,2020S,ACCTB202001,2020SBU ACCTB202001,10308884,CAS,POLS,BA,SO,U,E,,,,19.0,19463.0,833.0,0.0,-8500,0,-6000,0.0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1068.21,305.05,3204.63,915.15,-447.37,-315.79


In [134]:
fy20 = fy20[['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT',
            'CRS_AT_HRS', 'CRS_COL', 'NET_CRS_REVENUE']]
fy20.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE
0,2020S,ACCTB202001,10295572,MA,MUIN,BS,SR,U,E,3.0,BU,1080.96
1,2020S,ACCTB202001,10310914,BU,MGT,BBA,SO,U,E,3.0,BU,1219.20
2,2020S,ACCTB202001,10309131,BU,GSB,UND,SO,U,E,3.0,BU,1069.20
3,2020S,ACCTB202001,10307124,BU,MGT,BBA,SO,U,E,3.0,BU,723.99
4,2020S,ACCTB202001,10308884,CAS,POLS,BA,SO,U,E,3.0,BU,915.15


Remove majors that have been discontinued. No point projecting over them. See file Student Ratios by Majors.xlsx in FY20, sheet FY18.

In [135]:
fy20 = fy20.merge(crosswalk, on='STU_MAJ', how='left', validate='m:1')

In [136]:
fy20.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE,Department,College
0,2020S,ACCTB202001,10295572,MA,MUIN,BS,SR,U,E,3.0,BU,1080.96,Music Industry,CMM
1,2020S,ACCTB202001,10310914,BU,MGT,BBA,SO,U,E,3.0,BU,1219.20,Management,BU
2,2020S,ACCTB202001,10309131,BU,GSB,UND,SO,U,E,3.0,BU,1069.20,BU,BU
3,2020S,ACCTB202001,10307124,BU,MGT,BBA,SO,U,E,3.0,BU,723.99,Management,BU
4,2020S,ACCTB202001,10308884,CAS,POLS,BA,SO,U,E,3.0,BU,915.15,Political Science,CAS


In [137]:
fy20['TERM'][pd.isna(fy20['Department'])].count()

0

In [138]:
fy20['NET_CRS_REVENUE'].sum()

62327629.269999996

In [139]:
fy20.to_excel('../3_res/FY20_Revenues.xlsx', index=False)

In [140]:
fy18.columns

Index(['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'CRS_AT_HRS', 'CRS_COL', 'NET_CRS_REVENUE', 'Department', 'College'], dtype='object')

In [141]:
fy19.columns

Index(['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'CRS_AT_HRS', 'CRS_COL', 'NET_CRS_REVENUE', 'Department', 'College'], dtype='object')

In [142]:
fy20.columns

Index(['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'CRS_AT_HRS', 'CRS_COL', 'NET_CRS_REVENUE', 'Department', 'College'], dtype='object')

In [143]:
three_yr_rev = fy18.append(fy19, ignore_index=True)
three_yr_rev = three_yr_rev.append(fy20, ignore_index=True)
three_yr_rev.TERM.value_counts(dropna=False)

2019F    21444
2018F    21263
2020S    20631
2019S    20101
2017F    19451
2018S    18444
Name: TERM, dtype: int64

In [144]:
three_yr_rev.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,NET_CRS_REVENUE,Department,College
0,2018S,NURSG805W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50,Nursing,CNH
1,2018S,NURSG835W02,10002754,CNH,FNP,MSN,GFR,G,E,3.0,CNH,2566.50,Nursing,CNH
2,2018S,HISTT122004,10015160,CAS,NDRA,ND,NDU,U,E,3.0,CAS,401.49,CAS,CAS
3,2018S,DECSB893001,10020635,BU,NDGB,ND,NDG,G,E,3.0,BU,56.49,BU,BU
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,3.0,MA,3620.49,English,CAS


In [145]:
revenue = three_yr_rev.groupby(['TERM', 'CRS_ID'])[['CRS_AT_HRS', 'NET_CRS_REVENUE']].sum()
revenue.reset_index(inplace=True)

enrollment = three_yr_rev.groupby(['TERM', 'CRS_ID'])[['CWID']].count()
enrollment.reset_index(inplace=True)

revenue = revenue.merge(enrollment, on=['TERM', 'CRS_ID'], how='left', validate='1:1')

revenue.head()

,TERM,CRS_ID,CRS_AT_HRS,NET_CRS_REVENUE,CWID
0,2017F,ACCTB202001,66.0,30068.40,22
1,2017F,ACCTB202002,78.0,44291.79,26
2,2017F,ACCTB202003,78.0,56948.46,26
3,2017F,ACCTB203001,66.0,41433.75,22
4,2017F,ACCTB203002,48.0,28937.64,16


In [146]:
cat_vars = three_yr_rev.drop(['CRS_AT_HRS', 'NET_CRS_REVENUE'], axis = 1)
cat_vars.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_COL,Department,College
0,2018S,NURSG805W02,10002754,CNH,FNP,MSN,GFR,G,E,CNH,Nursing,CNH
1,2018S,NURSG835W02,10002754,CNH,FNP,MSN,GFR,G,E,CNH,Nursing,CNH
2,2018S,HISTT122004,10015160,CAS,NDRA,ND,NDU,U,E,CAS,CAS,CAS
3,2018S,DECSB893001,10020635,BU,NDGB,ND,NDG,G,E,BU,BU,BU
4,2018S,MUGNO268W02,10038247,CAS,ENGL,BA,SR,U,A,MA,English,CAS


In [147]:
cat_vars.columns

Index(['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT', 'CRS_COL', 'Department', 'College'], dtype='object')

In [148]:
cat_vars['College'][pd.isna(cat_vars['College'])]='Other'
cat_vars['College'].value_counts(dropna=False)

CAS      54909
CMM      32088
BU       17066
LAW       9291
CNH       7138
Other      842
Name: College, dtype: int64

In [149]:
cat_vars['STU_COHORT'][(cat_vars['STU_COHORT']=='C') | (cat_vars['STU_COHORT']=='B') | (cat_vars['STU_COHORT']=='D')]='Other'
cat_vars['STU_COHORT'].value_counts(dropna=False)

E        106274
G         12627
H          1583
A           581
Other       269
Name: STU_COHORT, dtype: int64

In [150]:
cat_vars.drop(['STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'CRS_COL'], axis=1, inplace=True)
cat_vars.head()

,TERM,CRS_ID,CWID,STU_LEV,STU_COHORT,Department,College
0,2018S,NURSG805W02,10002754,G,E,Nursing,CNH
1,2018S,NURSG835W02,10002754,G,E,Nursing,CNH
2,2018S,HISTT122004,10015160,U,E,CAS,CAS
3,2018S,DECSB893001,10020635,G,E,BU,BU
4,2018S,MUGNO268W02,10038247,U,A,English,CAS


In [151]:
cat_vars.columns

Index(['TERM', 'CRS_ID', 'CWID', 'STU_LEV', 'STU_COHORT', 'Department', 'College'], dtype='object')

In [152]:
cat_vars.rename(columns={'Department':'STU_DEPT', 'College':'STU_COL'}, inplace=True)
student_ind = pd.get_dummies(cat_vars, columns=['STU_LEV', 'STU_COHORT', 'STU_DEPT', 'STU_COL'])
student_ind.head()

,TERM,CRS_ID,CWID,STU_LEV_C,STU_LEV_D,STU_LEV_G,STU_LEV_P,STU_LEV_U,STU_COHORT_A,STU_COHORT_E,STU_COHORT_G,STU_COHORT_H,STU_COHORT_Other,STU_DEPT_Accounting,STU_DEPT_BU,STU_DEPT_Biology,STU_DEPT_CAS,STU_DEPT_Chemistry,STU_DEPT_City College,STU_DEPT_Communication,STU_DEPT_Computer Science,STU_DEPT_Counseling,STU_DEPT_Criminal Justice,STU_DEPT_Design,STU_DEPT_Economics,STU_DEPT_English,STU_DEPT_Environment,STU_DEPT_Exchange,STU_DEPT_Finance,STU_DEPT_History,STU_DEPT_LIM,STU_DEPT_Languages,STU_DEPT_Law,STU_DEPT_Management,STU_DEPT_Marketing,STU_DEPT_Math,STU_DEPT_Music,STU_DEPT_Music Industry,STU_DEPT_Nursing,STU_DEPT_Philosophy,STU_DEPT_Physics,STU_DEPT_Political Science,STU_DEPT_Psychology,STU_DEPT_Religious Studies,STU_DEPT_Sociology,STU_DEPT_Studio Art,STU_DEPT_Teacher Education,STU_DEPT_Theatre,STU_COL_BU,STU_COL_CAS,STU_COL_CMM,STU_COL_CNH,STU_COL_LAW,STU_COL_Other
0,2018S,NURSG805W02,10002754,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,2018S,NURSG835W02,10002754,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2018S,HISTT122004,10015160,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2018S,DECSB893001,10020635,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2018S,MUGNO268W02,10038247,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [153]:
student_ind.drop('CWID', axis=1, inplace=True)
student_ind = student_ind.groupby(['TERM', 'CRS_ID']).sum()

In [154]:
student_ind.reset_index(inplace=True)
student_ind.head()

,TERM,CRS_ID,STU_LEV_C,STU_LEV_D,STU_LEV_G,STU_LEV_P,STU_LEV_U,STU_COHORT_A,STU_COHORT_E,STU_COHORT_G,STU_COHORT_H,STU_COHORT_Other,STU_DEPT_Accounting,STU_DEPT_BU,STU_DEPT_Biology,STU_DEPT_CAS,STU_DEPT_Chemistry,STU_DEPT_City College,STU_DEPT_Communication,STU_DEPT_Computer Science,STU_DEPT_Counseling,STU_DEPT_Criminal Justice,STU_DEPT_Design,STU_DEPT_Economics,STU_DEPT_English,STU_DEPT_Environment,STU_DEPT_Exchange,STU_DEPT_Finance,STU_DEPT_History,STU_DEPT_LIM,STU_DEPT_Languages,STU_DEPT_Law,STU_DEPT_Management,STU_DEPT_Marketing,STU_DEPT_Math,STU_DEPT_Music,STU_DEPT_Music Industry,STU_DEPT_Nursing,STU_DEPT_Philosophy,STU_DEPT_Physics,STU_DEPT_Political Science,STU_DEPT_Psychology,STU_DEPT_Religious Studies,STU_DEPT_Sociology,STU_DEPT_Studio Art,STU_DEPT_Teacher Education,STU_DEPT_Theatre,STU_COL_BU,STU_COL_CAS,STU_COL_CMM,STU_COL_CNH,STU_COL_LAW,STU_COL_Other
0,2017F,ACCTB202001,0.0,0.0,0.0,0.0,22.0,0.0,22.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,0.0,3.0,0.0,0.0,0.0
1,2017F,ACCTB202002,0.0,0.0,0.0,0.0,26.0,0.0,26.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0,10.0,0.0,0.0,1.0
2,2017F,ACCTB202003,0.0,0.0,0.0,0.0,26.0,0.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,5.0,0.0,0.0,0.0
3,2017F,ACCTB203001,0.0,0.0,0.0,0.0,22.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0
4,2017F,ACCTB203002,0.0,0.0,0.0,0.0,16.0,1.0,15.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,1.0,0.0,0.0,0.0,0.0


In [155]:
revenue = revenue.merge(student_ind, on=['TERM', 'CRS_ID'], how='left', validate='1:1')
revenue.head()

,TERM,CRS_ID,CRS_AT_HRS,NET_CRS_REVENUE,CWID,STU_LEV_C,STU_LEV_D,STU_LEV_G,STU_LEV_P,STU_LEV_U,STU_COHORT_A,STU_COHORT_E,STU_COHORT_G,STU_COHORT_H,STU_COHORT_Other,STU_DEPT_Accounting,STU_DEPT_BU,STU_DEPT_Biology,STU_DEPT_CAS,STU_DEPT_Chemistry,STU_DEPT_City College,STU_DEPT_Communication,STU_DEPT_Computer Science,STU_DEPT_Counseling,STU_DEPT_Criminal Justice,STU_DEPT_Design,STU_DEPT_Economics,STU_DEPT_English,STU_DEPT_Environment,STU_DEPT_Exchange,STU_DEPT_Finance,STU_DEPT_History,STU_DEPT_LIM,STU_DEPT_Languages,STU_DEPT_Law,STU_DEPT_Management,STU_DEPT_Marketing,STU_DEPT_Math,STU_DEPT_Music,STU_DEPT_Music Industry,STU_DEPT_Nursing,STU_DEPT_Philosophy,STU_DEPT_Physics,STU_DEPT_Political Science,STU_DEPT_Psychology,STU_DEPT_Religious Studies,STU_DEPT_Sociology,STU_DEPT_Studio Art,STU_DEPT_Teacher Education,STU_DEPT_Theatre,STU_COL_BU,STU_COL_CAS,STU_COL_CMM,STU_COL_CNH,STU_COL_LAW,STU_COL_Other
0,2017F,ACCTB202001,66.0,30068.40,22,0.0,0.0,0.0,0.0,22.0,0.0,22.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,0.0,3.0,0.0,0.0,0.0
1,2017F,ACCTB202002,78.0,44291.79,26,0.0,0.0,0.0,0.0,26.0,0.0,26.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0,10.0,0.0,0.0,1.0
2,2017F,ACCTB202003,78.0,56948.46,26,0.0,0.0,0.0,0.0,26.0,0.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,5.0,0.0,0.0,0.0
3,2017F,ACCTB203001,66.0,41433.75,22,0.0,0.0,0.0,0.0,22.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0
4,2017F,ACCTB203002,48.0,28937.64,16,0.0,0.0,0.0,0.0,16.0,1.0,15.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,1.0,0.0,0.0,0.0,0.0


In [156]:
revenue.to_excel('../3_res/Students for merge.xlsx', index = False)

# FY 21 data

In [157]:
fy21 = pd.read_excel('../1_data/Students_Courses Fall 2020.xlsx')
fy21.head()

,CRS_ID,RTCRS_KEY,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,STU_SEC_COL,STU_SEC_MAJ,STU_SEC_DEG,SEM_HOURS,TUITSEM1,FEES,MFEES,LOY_SCHOL,LOY_ENDOW,LOY_DISC1,LOY_DISC2,LOY_DISC3,REDUC_DISC,REL_DISC,TUIT_WAIV,CRS_ID.1,FIN_AT_HRS,CRS_AT_HRS,STATUS,CRS_COL,AOC,EXCH_STA,CRS_SECTION,CRS_AREA,CRS_SUBJ,CRS_LEV,COURS_LEV,CRHR_CHARGE,NET_CRHR_CHARGE,GROSS_CRS_REV,NET_CRS_REVENUE,SCHOL_CRHR_CHG,DISC_CRHR_CHG
0,ACCTB202001,2020FBU ACCTB202001,10322419,BU,INTB,BBA,SO,U,E,,,,18.0,20144,870.0,1824.0,-9000,0,-8050,0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1167.44,220.22,3502.32,660.66,-500.0,-447.22
1,ACCTB202001,2020FBU ACCTB202001,10310119,BU,MGT,BBA,JR,U,E,,,,18.0,20144,870.0,250.0,0,0,0,0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1167.44,1167.44,3502.32,3502.32,0.0,0.00
2,ACCTB202001,2020FBU ACCTB202001,10315670,BU,FIN,BBA,SO,U,E,,,,18.0,20144,870.0,250.0,0,0,0,0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1167.44,1167.44,3502.32,3502.32,0.0,0.00
3,ACCTB202001,2020FBU ACCTB202001,10318688,BU,MKT,BBA,SO,U,E,,,,16.0,20144,870.0,250.0,0,0,0,0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1313.38,1313.38,3940.14,3940.14,0.0,0.00
4,ACCTB202001,2020FBU ACCTB202001,10315050,BU,FIN,BBA,SO,U,E,,,,15.0,20144,870.0,250.0,0,0,0,0,0,0.0,0,0,ACCTB202001,3.0,3.0,E,BU,,,001,B202,ACCT,202,U,1400.93,1400.93,4202.79,4202.79,0.0,0.00


In [158]:
fy21['TERM']='2020F'
fy21.groupby(['CWID', 'CRS_ID', 'TERM']).ngroups - fy21['CWID'].count()

0

In [159]:
fy21 = fy21[['TERM', 'CRS_ID', 'CWID', 'STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'STU_LEV', 'STU_COHORT',
            'CRS_AT_HRS', 'CRS_COL']]
fy21.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL
0,2020F,ACCTB202001,10322419,BU,INTB,BBA,SO,U,E,3.0,BU
1,2020F,ACCTB202001,10310119,BU,MGT,BBA,JR,U,E,3.0,BU
2,2020F,ACCTB202001,10315670,BU,FIN,BBA,SO,U,E,3.0,BU
3,2020F,ACCTB202001,10318688,BU,MKT,BBA,SO,U,E,3.0,BU
4,2020F,ACCTB202001,10315050,BU,FIN,BBA,SO,U,E,3.0,BU


In [160]:
fy21 = fy21.merge(crosswalk, on='STU_MAJ', how='left', validate='m:1')

In [161]:
fy21.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,Department,College
0,2020F,ACCTB202001,10322419,BU,INTB,BBA,SO,U,E,3.0,BU,Management,BU
1,2020F,ACCTB202001,10310119,BU,MGT,BBA,JR,U,E,3.0,BU,Management,BU
2,2020F,ACCTB202001,10315670,BU,FIN,BBA,SO,U,E,3.0,BU,Finance,BU
3,2020F,ACCTB202001,10318688,BU,MKT,BBA,SO,U,E,3.0,BU,Marketing,BU
4,2020F,ACCTB202001,10315050,BU,FIN,BBA,SO,U,E,3.0,BU,Finance,BU


In [162]:
fy21['TERM'][pd.isna(fy21['Department'])].count()

402

In [163]:
#fy21['crs_3'] = fy21['CRS_ID'].astype(str).str[0:3]
#fy21['crs_4'] = fy21['CRS_ID'].astype(str).str[0:4]

#fy21.head()

In [164]:
fy21['STU_MAJ'][pd.isna(fy21['Department'])].value_counts(dropna=False)

EXPL    203
MKCM     62
ACCT     58
NEUP     21
HLA      14
NEUB     12
CSEC     11
EVL       8
NEUS      6
MATC      6
NDES      1
Name: STU_MAJ, dtype: int64

In [165]:
fy21['Department'][fy21['STU_MAJ']=='ACCT']='Accounting'
fy21['College'][fy21['STU_MAJ']=='ACCT']='BU'
fy21['Department'][fy21['STU_MAJ']=='NEUP']='Psychology'
fy21['College'][fy21['STU_MAJ']=='NEUP']='CAS'
fy21['Department'][fy21['STU_MAJ']=='NEUB']='Biology'
fy21['College'][fy21['STU_MAJ']=='NEUB']='CAS'
fy21['Department'][fy21['STU_MAJ']=='NEUS']='Honors'
fy21['College'][fy21['STU_MAJ']=='NEUS']='CAS'
fy21['Department'][fy21['STU_MAJ']=='CSEC']='Computer Science'
fy21['College'][fy21['STU_MAJ']=='CSEC']='CAS'
fy21['Department'][fy21['STU_MAJ']=='MATC']='Math'
fy21['College'][fy21['STU_MAJ']=='MATC']='CAS'

In [166]:
fy21['STU_MAJ'][pd.isna(fy21['Department'])].value_counts(dropna=False)

EXPL    203
MKCM     62
HLA      14
EVL       8
NDES      1
Name: STU_MAJ, dtype: int64

In [167]:
print(fy21['CWID'][pd.isna(fy21['Department'])].nunique())
fy21['CWID'][pd.isna(fy21['Department'])].nunique()/fy21['CWID'].nunique() #1.4% of students - drop these

#EXPL are exploratory - yet to decide. Can't include anyway - so ok

63


0.013840070298769772

In [168]:
fy21 = fy21[pd.notna(fy21['Department'])]

In [169]:
fy21.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,Department,College
0,2020F,ACCTB202001,10322419,BU,INTB,BBA,SO,U,E,3.0,BU,Management,BU
1,2020F,ACCTB202001,10310119,BU,MGT,BBA,JR,U,E,3.0,BU,Management,BU
2,2020F,ACCTB202001,10315670,BU,FIN,BBA,SO,U,E,3.0,BU,Finance,BU
3,2020F,ACCTB202001,10318688,BU,MKT,BBA,SO,U,E,3.0,BU,Marketing,BU
4,2020F,ACCTB202001,10315050,BU,FIN,BBA,SO,U,E,3.0,BU,Finance,BU


In [170]:
fy21.isnull().sum()

TERM          0
CRS_ID        0
CWID          0
STU_COL       0
STU_MAJ       0
STU_DEG       0
STU_CLASS     0
STU_LEV       0
STU_COHORT    0
CRS_AT_HRS    0
CRS_COL       0
Department    0
College       4
dtype: int64

In [171]:
fy21[pd.isna(fy21['College'])]

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,Department,College
7266,2020F,HISTQ234001,10331198,CAS,EXUP,EX,EXC,U,E,3.0,CAS,Exchange,NaN
7365,2020F,HISTQ250002,10331198,CAS,EXUP,EX,EXC,U,E,3.0,CAS,Exchange,NaN
8022,2020F,HISTT200002,10331198,CAS,EXUP,EX,EXC,U,E,3.0,CAS,Exchange,NaN
19297,2020F,POLSX259001,10331198,CAS,EXUP,EX,EXC,U,E,3.0,CAS,Exchange,NaN


In [172]:
fy21.College.value_counts(dropna=False)

CAS    11350
CMM     5256
BU      3427
LAW     1575
CNH     1209
NaN        4
Name: College, dtype: int64

In [173]:
fy21['College'][pd.isna(fy21['College'])]='CAS'
fy21.College.value_counts(dropna=False)

CAS    11354
CMM     5256
BU      3427
LAW     1575
CNH     1209
Name: College, dtype: int64

In [174]:
fy21.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_AT_HRS,CRS_COL,Department,College
0,2020F,ACCTB202001,10322419,BU,INTB,BBA,SO,U,E,3.0,BU,Management,BU
1,2020F,ACCTB202001,10310119,BU,MGT,BBA,JR,U,E,3.0,BU,Management,BU
2,2020F,ACCTB202001,10315670,BU,FIN,BBA,SO,U,E,3.0,BU,Finance,BU
3,2020F,ACCTB202001,10318688,BU,MKT,BBA,SO,U,E,3.0,BU,Marketing,BU
4,2020F,ACCTB202001,10315050,BU,FIN,BBA,SO,U,E,3.0,BU,Finance,BU


In [175]:
fy21.groupby(['CRS_ID', 'CWID']).ngroups - fy21['TERM'].count()

0

In [176]:
rev21 = fy21.groupby('CRS_ID')[['CRS_AT_HRS']].sum()
rev21.reset_index(inplace=True)

enr21 = fy21.groupby(['CRS_ID'])[['CWID']].count()
enr21.reset_index(inplace=True)

rev21 = rev21.merge(enr21, on=['CRS_ID'], how='left', validate='1:1')
rev21.rename(columns={'CWID':'Enrollment'}, inplace=True)
rev21.head()

,CRS_ID,CRS_AT_HRS,Enrollment
0,ACCTB202001,114.0,38
1,ACCTB202002,72.0,24
2,ACCTB202003,120.0,40
3,ACCTB203001,123.0,41
4,ACCTB300001,48.0,16


In [177]:
cat21 = fy21.drop(['CRS_AT_HRS'], axis = 1)
cat21.head()

,TERM,CRS_ID,CWID,STU_COL,STU_MAJ,STU_DEG,STU_CLASS,STU_LEV,STU_COHORT,CRS_COL,Department,College
0,2020F,ACCTB202001,10322419,BU,INTB,BBA,SO,U,E,BU,Management,BU
1,2020F,ACCTB202001,10310119,BU,MGT,BBA,JR,U,E,BU,Management,BU
2,2020F,ACCTB202001,10315670,BU,FIN,BBA,SO,U,E,BU,Finance,BU
3,2020F,ACCTB202001,10318688,BU,MKT,BBA,SO,U,E,BU,Marketing,BU
4,2020F,ACCTB202001,10315050,BU,FIN,BBA,SO,U,E,BU,Finance,BU


In [178]:
cat21['STU_COHORT'].value_counts(dropna=False)

E    20033
G     2395
H      225
C       82
A       82
D        4
Name: STU_COHORT, dtype: int64

In [179]:
cat21['STU_COHORT'][(cat21['STU_COHORT']=='C') | (cat21['STU_COHORT']=='B') | (cat21['STU_COHORT']=='D')]='Other'
cat21['STU_COHORT'].value_counts(dropna=False)

E        20033
G         2395
H          225
Other       86
A           82
Name: STU_COHORT, dtype: int64

In [180]:
cat21.drop(['STU_COL', 'STU_MAJ', 'STU_DEG', 'STU_CLASS', 'CRS_COL'], axis=1, inplace=True)
cat21.head()

,TERM,CRS_ID,CWID,STU_LEV,STU_COHORT,Department,College
0,2020F,ACCTB202001,10322419,U,E,Management,BU
1,2020F,ACCTB202001,10310119,U,E,Management,BU
2,2020F,ACCTB202001,10315670,U,E,Finance,BU
3,2020F,ACCTB202001,10318688,U,E,Marketing,BU
4,2020F,ACCTB202001,10315050,U,E,Finance,BU


In [181]:
cat21.rename(columns={'Department':'STU_DEPT', 'College':'STU_COL'}, inplace=True)
stu21 = pd.get_dummies(cat21, columns=['STU_LEV', 'STU_COHORT', 'STU_DEPT', 'STU_COL'])
stu21.head()

,TERM,CRS_ID,CWID,STU_LEV_C,STU_LEV_D,STU_LEV_G,STU_LEV_P,STU_LEV_U,STU_COHORT_A,STU_COHORT_E,STU_COHORT_G,STU_COHORT_H,STU_COHORT_Other,STU_DEPT_Accounting,STU_DEPT_BU,STU_DEPT_Biology,STU_DEPT_CAS,STU_DEPT_Chemistry,STU_DEPT_Communication,STU_DEPT_Computer Science,STU_DEPT_Counseling,STU_DEPT_Criminal Justice,STU_DEPT_Design,STU_DEPT_Economics,STU_DEPT_English,STU_DEPT_Environment,STU_DEPT_Exchange,STU_DEPT_Finance,STU_DEPT_History,STU_DEPT_Honors,STU_DEPT_LIM,STU_DEPT_Languages,STU_DEPT_Law,STU_DEPT_Management,STU_DEPT_Marketing,STU_DEPT_Math,STU_DEPT_Music,STU_DEPT_Music Industry,STU_DEPT_Nursing,STU_DEPT_Philosophy,STU_DEPT_Physics,STU_DEPT_Political Science,STU_DEPT_Psychology,STU_DEPT_Religious Studies,STU_DEPT_Sociology,STU_DEPT_Studio Art,STU_DEPT_Teacher Education,STU_DEPT_Theatre,STU_COL_BU,STU_COL_CAS,STU_COL_CMM,STU_COL_CNH,STU_COL_LAW
0,2020F,ACCTB202001,10322419,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2020F,ACCTB202001,10310119,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2020F,ACCTB202001,10315670,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2020F,ACCTB202001,10318688,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2020F,ACCTB202001,10315050,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [182]:
stu21.drop('CWID', axis=1, inplace=True)
stu21 = stu21.groupby(['CRS_ID']).sum()

In [183]:
stu21.reset_index(inplace=True)
stu21.head()

,CRS_ID,STU_LEV_C,STU_LEV_D,STU_LEV_G,STU_LEV_P,STU_LEV_U,STU_COHORT_A,STU_COHORT_E,STU_COHORT_G,STU_COHORT_H,STU_COHORT_Other,STU_DEPT_Accounting,STU_DEPT_BU,STU_DEPT_Biology,STU_DEPT_CAS,STU_DEPT_Chemistry,STU_DEPT_Communication,STU_DEPT_Computer Science,STU_DEPT_Counseling,STU_DEPT_Criminal Justice,STU_DEPT_Design,STU_DEPT_Economics,STU_DEPT_English,STU_DEPT_Environment,STU_DEPT_Exchange,STU_DEPT_Finance,STU_DEPT_History,STU_DEPT_Honors,STU_DEPT_LIM,STU_DEPT_Languages,STU_DEPT_Law,STU_DEPT_Management,STU_DEPT_Marketing,STU_DEPT_Math,STU_DEPT_Music,STU_DEPT_Music Industry,STU_DEPT_Nursing,STU_DEPT_Philosophy,STU_DEPT_Physics,STU_DEPT_Political Science,STU_DEPT_Psychology,STU_DEPT_Religious Studies,STU_DEPT_Sociology,STU_DEPT_Studio Art,STU_DEPT_Teacher Education,STU_DEPT_Theatre,STU_COL_BU,STU_COL_CAS,STU_COL_CMM,STU_COL_CNH,STU_COL_LAW
0,ACCTB202001,0.0,0.0,0.0,0.0,38.0,0.0,38.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,14.0,9.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,32.0,3.0,3.0,0.0,0.0
1,ACCTB202002,0.0,0.0,0.0,0.0,24.0,0.0,24.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,20.0,3.0,1.0,0.0,0.0
2,ACCTB202003,0.0,0.0,0.0,0.0,40.0,0.0,40.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,13.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,32.0,6.0,2.0,0.0,0.0
3,ACCTB203001,0.0,0.0,0.0,0.0,41.0,0.0,41.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,19.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0
4,ACCTB300001,0.0,0.0,0.0,0.0,16.0,0.0,16.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,0.0,0.0


In [184]:
rev21 = rev21.merge(stu21, on=['CRS_ID'], how='left', validate='1:1')
rev21.head()

,CRS_ID,CRS_AT_HRS,Enrollment,STU_LEV_C,STU_LEV_D,STU_LEV_G,STU_LEV_P,STU_LEV_U,STU_COHORT_A,STU_COHORT_E,STU_COHORT_G,STU_COHORT_H,STU_COHORT_Other,STU_DEPT_Accounting,STU_DEPT_BU,STU_DEPT_Biology,STU_DEPT_CAS,STU_DEPT_Chemistry,STU_DEPT_Communication,STU_DEPT_Computer Science,STU_DEPT_Counseling,STU_DEPT_Criminal Justice,STU_DEPT_Design,STU_DEPT_Economics,STU_DEPT_English,STU_DEPT_Environment,STU_DEPT_Exchange,STU_DEPT_Finance,STU_DEPT_History,STU_DEPT_Honors,STU_DEPT_LIM,STU_DEPT_Languages,STU_DEPT_Law,STU_DEPT_Management,STU_DEPT_Marketing,STU_DEPT_Math,STU_DEPT_Music,STU_DEPT_Music Industry,STU_DEPT_Nursing,STU_DEPT_Philosophy,STU_DEPT_Physics,STU_DEPT_Political Science,STU_DEPT_Psychology,STU_DEPT_Religious Studies,STU_DEPT_Sociology,STU_DEPT_Studio Art,STU_DEPT_Teacher Education,STU_DEPT_Theatre,STU_COL_BU,STU_COL_CAS,STU_COL_CMM,STU_COL_CNH,STU_COL_LAW
0,ACCTB202001,114.0,38,0.0,0.0,0.0,0.0,38.0,0.0,38.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,14.0,9.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,32.0,3.0,3.0,0.0,0.0
1,ACCTB202002,72.0,24,0.0,0.0,0.0,0.0,24.0,0.0,24.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,20.0,3.0,1.0,0.0,0.0
2,ACCTB202003,120.0,40,0.0,0.0,0.0,0.0,40.0,0.0,40.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,13.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,32.0,6.0,2.0,0.0,0.0
3,ACCTB203001,123.0,41,0.0,0.0,0.0,0.0,41.0,0.0,41.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,19.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0
4,ACCTB300001,48.0,16,0.0,0.0,0.0,0.0,16.0,0.0,16.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,0.0,0.0


In [185]:
rev21.to_excel('../3_res/FY21 Students for merge.xlsx', index = False)

In [186]:
cat21.to_excel('../3_res/FY21 Unique Course IDs.xlsx', index=False)